In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.inspection import permutation_importance

# Prerprocessing 

In [2]:
df = pd.read_table("hw.0402.tsv")
df.head(5)

,bead_batch,pert_dose,pert_dose_unit,pert_time,pert_time_unit,nsample,tas,pert_id,sig_id,pert_type,cell_iname,det_wells,cmap_name
0,b25,0.213856,uM,24.0,h,2,0.112658,BRD-K92758126,REP.A028_YAPC_24H:C04,trt_cp,YAPC,C04,MD-920
1,b23,1.505850,uM,24.0,h,3,0.127106,BRD-K64874225,REP.A017_YAPC_24H:O08,trt_cp,YAPC,O08,NSC-4644
2,b23,1.000000,uM,24.0,h,3,0.356014,BRD-A68281735,REP.A023_YAPC_24H:D01,trt_cp,YAPC,D01,REV-5901
3,b25,1.119140,uM,24.0,h,3,0.087267,BRD-K01029106,REP.A012_A549_24H:E15,trt_cp,A549,E15,picolinic-acid
4,b24,3.848900,uM,24.0,h,3,0.053473,BRD-K15170068,REP.A010_MCF7_24H:M20,trt_cp,MCF7,M20,swainsonine


Let's explore the pert_id and sig_id for unique values. Here we have a lot of symbols at these 2 columns, so it is better to replace these ids with numerical values like:

- BRD-K92758126 is 1 for pert_id.

- REP.A028_YAPC_24H:C04 is 1 for sig_id.

For pert_id we do have 1570 unique values so we encode it into 1570 numerical values, for sig_id all values are unique. This type of preproprecessing saves a lot of memory, because if we apply one hot encoding via get_dummies from pandas or one hot encoding from numpy we can run out of local memory whether working on colab or kaggle.

In [3]:
df.shape

(106951, 13)

In [4]:
len(df.pert_id.unique())

1570

In [5]:
len(df.sig_id.unique())

106951

For our purposes we have used factorize

In [6]:
df['pert_id'] = pd.factorize(df.pert_id)[0] + 1
df['sig_id'] = pd.factorize(df.sig_id)[0] + 1

In [7]:
df.head(5)

,bead_batch,pert_dose,pert_dose_unit,pert_time,pert_time_unit,nsample,tas,pert_id,sig_id,pert_type,cell_iname,det_wells,cmap_name
0,b25,0.213856,uM,24.0,h,2,0.112658,1,1,trt_cp,YAPC,C04,MD-920
1,b23,1.505850,uM,24.0,h,3,0.127106,2,2,trt_cp,YAPC,O08,NSC-4644
2,b23,1.000000,uM,24.0,h,3,0.356014,3,3,trt_cp,YAPC,D01,REV-5901
3,b25,1.119140,uM,24.0,h,3,0.087267,4,4,trt_cp,A549,E15,picolinic-acid
4,b24,3.848900,uM,24.0,h,3,0.053473,5,5,trt_cp,MCF7,M20,swainsonine


In [8]:
y_new_dat = df.tas
X_new_dat = df.drop(columns=['tas'], axis=1)

In [9]:
# df = df.drop(columns=["pert_id", 'sig_id'], axis=1)
# df.head(5)

In [10]:
columns = ('bead_batch pert_dose pert_dose_unit pert_time pert_time_unit nsample tas '
           'pert_id	sig_id pert_type cell_iname det_wells  cmap_name')

numeric_indices = np.array([1, 3, 5, 6, 7, 8])
categorical_indices = np.array([0, 2, 4, 9, 10, 11, 12])

df.columns = columns.split() 

df = df.replace('?', np.nan)

df = df.dropna()

In [11]:
numeric_data = df[df.columns[numeric_indices]]

categorial_data = df[df.columns[categorical_indices]]
categorial_data.head()

,bead_batch,pert_dose_unit,pert_time_unit,pert_type,cell_iname,det_wells,cmap_name
0,b25,uM,h,trt_cp,YAPC,C04,MD-920
1,b23,uM,h,trt_cp,YAPC,O08,NSC-4644
2,b23,uM,h,trt_cp,YAPC,D01,REV-5901
3,b25,uM,h,trt_cp,A549,E15,picolinic-acid
4,b24,uM,h,trt_cp,MCF7,M20,swainsonine


In [12]:
numeric_data.head()

,pert_dose,pert_time,nsample,tas,pert_id,sig_id
0,0.213856,24.0,2,0.112658,1,1
1,1.505850,24.0,3,0.127106,2,2
2,1.000000,24.0,3,0.356014,3,3
3,1.119140,24.0,3,0.087267,4,4
4,3.848900,24.0,3,0.053473,5,5


In [13]:
y_new_dat = numeric_data.tas
X_new_dat = numeric_data.drop(columns=['tas'], axis=1)

One hot encoding for categorial_data

In [14]:
dummy_features = pd.get_dummies(categorial_data)

In [15]:
df_new = pd.concat([numeric_data, dummy_features], axis=1)
df_new.head()

,pert_dose,pert_time,nsample,tas,pert_id,sig_id,bead_batch_b22,bead_batch_b23,bead_batch_b24,bead_batch_b25,...,cmap_name_zaleplon,cmap_name_zaprinast,cmap_name_zardaverine,cmap_name_zidovudine,cmap_name_zileuton,cmap_name_ziprasidone,cmap_name_zofenopril-calcium,cmap_name_zolpidem,cmap_name_zonisamide,cmap_name_zosuquidar
0,0.213856,24.0,2,0.112658,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1.505850,24.0,3,0.127106,2,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.000000,24.0,3,0.356014,3,3,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.119140,24.0,3,0.087267,4,4,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,3.848900,24.0,3,0.053473,5,5,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
y = df_new['tas']
X = df_new.drop(columns="tas", axis=1)

# Train test split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, 
                                                    train_size=0.8,
                                                    random_state=42)

# Linear Regression

In [18]:
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [19]:
reg.score(X_train, y_train)

0.7717091368430036

In [20]:
y_pred = reg.predict(X_test)

# Calculating an error

In [21]:
mean_squared_error(y_train, reg.predict(X_train))

0.008247723472304744

In [22]:
mean_squared_error(y_test, y_pred)

0.008700499757589271

Here we see that error applying mse on train data is a little bit less than on test data. It is something to be expected as establishing a prediction on the same dataset that had been used for fitting might cause small overfitting.

In [23]:
reg1 = linear_model.LinearRegression()
reg1.fit(X_new_dat, y_new_dat)

LinearRegression()

# Cross Validation

we used Kfold to split data in 5 pieces and than cross_validate to train which is faster than straight looping through

In [24]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(X_train, y_train):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

Fold:1, Train set: 68448, Test set:17112
Fold:2, Train set: 68448, Test set:17112
Fold:3, Train set: 68448, Test set:17112
Fold:4, Train set: 68448, Test set:17112
Fold:5, Train set: 68448, Test set:17112


In [25]:
def rmse(score):
    rmse = np.sqrt(-score)
    print(f'rmse= {"{:.2f}".format(rmse)}')

In [26]:
output = cross_validate(linear_model.LinearRegression(), X_train, y_train, cv= kf, 
                        scoring="neg_mean_squared_error", return_estimator =True)

In [27]:
score = cross_val_score(linear_model.LinearRegression(), X_train, y_train, cv= kf, 
                        scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
rmse(score.mean())

Scores for each fold: [-0.00860501 -0.00901909 -0.00870252 -0.00858629 -0.00858617]
rmse= 0.09


# Feature importance

Using permutation_importance to value numerical features. Here I skipped encoded categorical features because there are so many of them and they consists of [0, 1] tensors. 

In [28]:
fe = permutation_importance(reg1, X_new_dat, y_new_dat)

In [29]:
fe

{'importances_mean': array([0.8350306 , 0.        , 0.01974225, 0.01836067, 0.03581953]),
 'importances_std': array([0.00372904, 0.        , 0.00022247, 0.00023222, 0.00059021]),
 'importances': array([[0.83521055, 0.83058931, 0.84129705, 0.83196739, 0.8360887 ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.0196032 , 0.02000929, 0.01965384, 0.01999469, 0.01945024],
        [0.01804309, 0.01842108, 0.01864001, 0.01855797, 0.01814121],
        [0.03681699, 0.03586026, 0.03533734, 0.03596465, 0.03511842]])}

# Feature engineering 

Now making 9 new features for products of numerical features and then concat it with dummies that we have got earlier.

In [30]:
numeric_data.head(5)

,pert_dose,pert_time,nsample,tas,pert_id,sig_id
0,0.213856,24.0,2,0.112658,1,1
1,1.505850,24.0,3,0.127106,2,2
2,1.000000,24.0,3,0.356014,3,3
3,1.119140,24.0,3,0.087267,4,4
4,3.848900,24.0,3,0.053473,5,5


In [31]:
numeric_data = numeric_data.drop(columns=["tas"], axis=1)
numeric_data.head(5)

,pert_dose,pert_time,nsample,pert_id,sig_id
0,0.213856,24.0,2,1,1
1,1.505850,24.0,3,2,2
2,1.000000,24.0,3,3,3
3,1.119140,24.0,3,4,4
4,3.848900,24.0,3,5,5


In [32]:
columns = ["pert_dose", "pert_time", "nsample", "pert_id", "sig_id"]

In [33]:
numeric_data["new_1"] = numeric_data["pert_dose"] * numeric_data["pert_time"]
numeric_data["new_2"] = numeric_data["pert_dose"] * numeric_data["nsample"]
numeric_data["new_3"] = numeric_data["pert_dose"] * numeric_data["pert_id"]
numeric_data["new_4"] = numeric_data["pert_dose"] * numeric_data["sig_id"]

numeric_data["new_5"] = numeric_data["pert_time"] * numeric_data["sig_id"]
numeric_data["new_6"] = numeric_data["pert_time"] * numeric_data["nsample"]
numeric_data["new_7"] = numeric_data["pert_time"] * numeric_data["pert_id"]

numeric_data["new_8"] = numeric_data["nsample"] * numeric_data["pert_id"]
numeric_data["new_9"] = numeric_data["nsample"] * numeric_data["sig_id"]

numeric_data["new_10"] = numeric_data["pert_id"] * numeric_data["sig_id"]

In [34]:
numeric_data.head(5)

,pert_dose,pert_time,nsample,pert_id,sig_id,new_1,new_2,new_3,new_4,new_5,new_6,new_7,new_8,new_9,new_10
0,0.213856,24.0,2,1,1,5.132544,0.427712,0.213856,0.213856,24.0,48.0,24.0,2,2,1
1,1.505850,24.0,3,2,2,36.140400,4.517550,3.011700,3.011700,48.0,72.0,48.0,6,6,4
2,1.000000,24.0,3,3,3,24.000000,3.000000,3.000000,3.000000,72.0,72.0,72.0,9,9,9
3,1.119140,24.0,3,4,4,26.859360,3.357420,4.476560,4.476560,96.0,72.0,96.0,12,12,16
4,3.848900,24.0,3,5,5,92.373600,11.546700,19.244500,19.244500,120.0,72.0,120.0,15,15,25


In [35]:
df_new_after = pd.concat([numeric_data, dummy_features], axis=1)
df_new_after.head()

,pert_dose,pert_time,nsample,pert_id,sig_id,new_1,new_2,new_3,new_4,new_5,...,cmap_name_zaleplon,cmap_name_zaprinast,cmap_name_zardaverine,cmap_name_zidovudine,cmap_name_zileuton,cmap_name_ziprasidone,cmap_name_zofenopril-calcium,cmap_name_zolpidem,cmap_name_zonisamide,cmap_name_zosuquidar
0,0.213856,24.0,2,1,1,5.132544,0.427712,0.213856,0.213856,24.0,...,0,0,0,0,0,0,0,0,0,0
1,1.505850,24.0,3,2,2,36.140400,4.517550,3.011700,3.011700,48.0,...,0,0,0,0,0,0,0,0,0,0
2,1.000000,24.0,3,3,3,24.000000,3.000000,3.000000,3.000000,72.0,...,0,0,0,0,0,0,0,0,0,0
3,1.119140,24.0,3,4,4,26.859360,3.357420,4.476560,4.476560,96.0,...,0,0,0,0,0,0,0,0,0,0
4,3.848900,24.0,3,5,5,92.373600,11.546700,19.244500,19.244500,120.0,...,0,0,0,0,0,0,0,0,0,0


Now apply split for new dataset and making cross validation for kfolds

In [36]:
y_new_after = y
X_new_after = df_new

In [37]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new_after.values, y_new_after.values, 
                                                    train_size=0.8,
                                                    random_state=42)

In [38]:
output_1 = cross_validate(linear_model.LinearRegression(), X_train_new, y_train_new, cv= kf, 
                        scoring="neg_mean_squared_error", return_estimator =True)

In [39]:
score = cross_val_score(linear_model.LinearRegression(), X_train_new, y_train_new, cv= kf, 
                        scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
rmse(score.mean())

Scores for each fold: [-2.12857837e-23 -5.52615119e-22 -9.55986214e-23 -5.40578800e-21
 -3.85087414e-22]
rmse= 0.00


Best model for kf=1

# Testing model

In [40]:
mean_squared_error(y_test, linear_model.LinearRegression().fit(X_train_new, y_train_new).predict(X_test_new))

7.27817651298152e-22

The feature might capture product of pert_dose and pert_time